In [1]:
%cd /hdd/yuchen/pipeline/training_pipeline

/hdd/yuchen/pipeline/training_pipeline


In [2]:
from torch_utils.engine import ( train_one_epoch, evaluate, utils)
from torch.utils.data import ( distributed, RandomSampler, SequentialSampler)
from datasets import (
    create_train_dataset, create_valid_dataset, 
    create_train_loader, create_valid_loader)
from models.create_fasterrcnn_model import create_model
from utils.general import (
    set_training_dir, Averager, 
    save_model, save_loss_plot,
    show_tranformed_image,
    save_mAP, save_model_state, SaveBestModel,
    yaml_save, init_seeds)
from utils.logging import (
    set_log, coco_log,set_summary_writer, tensorboard_loss_log,  tensorboard_map_log,
    csv_log, wandb_log,  wandb_save_model, wandb_init)

import torch
import argparse
import yaml
import numpy as np
import torchinfo
import os

In [4]:
parse_opt = {
    'model' : 'fasterrcnn_resnet50_fpn_v2',
    'data' : '',
    'device' : 'cuda:0',
    'epochs' : 5,
    'workers' : 4,
    'batch' : 2,
    'lr' : 0.001,
    'imgsz' : 224,
    'name' : '',
    'vis_transformed' : False,
    'mosaic' : 0.0,
    'use_train_aug' : False,
    'cosine_annealing' : True,  # use cosine annealing warm restarts
    'weights' : None,  # load weights directly from files
    'resume_training' : False,
    'square_training' : True,  # 'Resize images to square shape
    'sync_bn' : True,  # sync batch norm
    'amp' : True,  # use automatic mixed precision
    'seed' : 42,
    'project_dir' : None
}
args = parse_opt

In [5]:
args['epochs'] = 90
args['imgsz'] = 224
args['device'] = 'cuda:1'
args['model'] = 'fasterrcnn_swin'

args['name'] = 'land4sensor_training'
args['batch'] = 8
args['data'] = 'data_configs/land4sensor.yaml'

In [6]:
torch.multiprocessing.set_sharing_strategy('file_system')

RANK = int(os.getenv('RANK', -1))

# For same annotation colors each time.
np.random.seed(42)

with open(args['data']) as file:
    data_configs = yaml.safe_load(file)

init_seeds(args['seed'] + 1 + RANK, deterministic=True)
    
    # Settings/parameters/constants.
TRAIN_DIR_IMAGES = os.path.normpath(data_configs['TRAIN_DIR_IMAGES'])
TRAIN_DIR_LABELS = os.path.normpath(data_configs['TRAIN_DIR_LABELS'])
VALID_DIR_IMAGES = os.path.normpath(data_configs['VALID_DIR_IMAGES'])
VALID_DIR_LABELS = os.path.normpath(data_configs['VALID_DIR_LABELS'])
CLASSES = data_configs['CLASSES']
NUM_CLASSES = data_configs['NC']
NUM_WORKERS = args['workers']
DEVICE = torch.device(args['device'])
print("device",DEVICE)
NUM_EPOCHS = args['epochs']
SAVE_VALID_PREDICTIONS = data_configs['SAVE_VALID_PREDICTION_IMAGES']
BATCH_SIZE = args['batch']
VISUALIZE_TRANSFORMED_IMAGES = args['vis_transformed']
OUT_DIR = set_training_dir(args['name'], args['project_dir'])
COLORS = np.random.uniform(0, 1, size=(len(CLASSES), 3))
SCALER = torch.cuda.amp.GradScaler() if args['amp'] else None

device cuda:1


In [7]:
set_log(OUT_DIR)
writer = set_summary_writer(OUT_DIR)

yaml_save(file_path=os.path.join(OUT_DIR, 'opt.yaml'), data=args)

    # Model configurations
IMAGE_SIZE = args['imgsz']
    
train_dataset = create_train_dataset(
        TRAIN_DIR_IMAGES, 
        TRAIN_DIR_LABELS,
        IMAGE_SIZE, 
        CLASSES,
        use_train_aug=args['use_train_aug'],
        mosaic=args['mosaic'],
        square_training=args['square_training']
    )
valid_dataset = create_valid_dataset(
        VALID_DIR_IMAGES, 
        VALID_DIR_LABELS, 
        IMAGE_SIZE, 
        CLASSES,
        square_training=args['square_training']
    )

train_sampler = RandomSampler(train_dataset)
valid_sampler = SequentialSampler(valid_dataset)

train_loader = create_train_loader(train_dataset, BATCH_SIZE, NUM_WORKERS, batch_sampler=train_sampler)
valid_loader = create_valid_loader(valid_dataset, BATCH_SIZE, NUM_WORKERS)

Checking Labels and images...


  0%|          | 0/1440 [00:00<?, ?it/s]

Checking Labels and images...


  0%|          | 0/309 [00:00<?, ?it/s]

In [8]:
train_loss_hist = Averager()
train_loss_list = []
loss_cls_list = []
loss_box_reg_list = []
loss_objectness_list = []
loss_rpn_list = []
train_loss_list_epoch = []
val_map_05 = []
val_map = []
start_epochs = 0

if args['weights'] is None:
    print('Building model from scratch or load weights from files')
    build_model = create_model[args['model']]
    model = build_model(num_classes=NUM_CLASSES, pretrained=True)

    model.transform.min_size = (args['imgsz'], )
    model = model.to(DEVICE)

    # print(model)

total_params = sum(p.numel() for p in model.parameters())
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=args['lr'], momentum=0.9, nesterov=True)
# optimizer = torch.optim.AdamW(params, lr=0.0001, weight_decay=0.0005)

if args['cosine_annealing']:
    # LR will be zero as we approach `steps` number of epochs each time.
    # If `steps = 5`, LR will slowly reduce to zero every 5 epochs.
    steps = NUM_EPOCHS + 10
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, 
        T_0=steps,
        T_mult=1,
        verbose=False
    )
else:
    scheduler = None

save_best_model = SaveBestModel()

Building model from scratch or load weights from files
current model setting: base_from_mae


/hdd/yuchen/anaconda3/envs/291k/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [ ]:
best_map = -np.inf
best_map_05 = -np.inf
patience = 10
ct = 0


for epoch in range(start_epochs, NUM_EPOCHS):
            
    train_loss_hist.reset()

    _, batch_loss_list, \
            batch_loss_cls_list, \
            batch_loss_box_reg_list, \
            batch_loss_objectness_list, \
            batch_loss_rpn_list = train_one_epoch(
            model, 
            optimizer, 
            train_loader, 
            DEVICE, 
            epoch, 
            train_loss_hist,
            print_freq=100,
                
            scheduler=scheduler,
            scaler=SCALER
        )

    stats, val_pred_image = evaluate(
            model, 
            valid_loader, 
            device=DEVICE,
            save_valid_preds=SAVE_VALID_PREDICTIONS,
            out_dir=OUT_DIR,
            classes=CLASSES,
            colors=COLORS
        )

    # Append the current epoch's batch-wise losses to the `train_loss_list`.
    train_loss_list.extend(batch_loss_list)
    loss_cls_list.append(np.mean(np.array(batch_loss_cls_list,)))
    loss_box_reg_list.append(np.mean(np.array(batch_loss_box_reg_list)))
    loss_objectness_list.append(np.mean(np.array(batch_loss_objectness_list)))
    loss_rpn_list.append(np.mean(np.array(batch_loss_rpn_list)))

    # Append curent epoch's average loss to `train_loss_list_epoch`.
    train_loss_list_epoch.append(train_loss_hist.value)
    val_map_05.append(stats[1])
    val_map.append(stats[0])

    print(stats[0],stats[1] )

    if stats[0] > best_map:
        best_map = stats[0]
        ct = 0
        torch.save({
                'epoch': 1,
                'model_state_dict': model.state_dict(),
                'data':  data_configs,
                'model_name': args['model'],
                }, f"{OUT_DIR}/best_model.pth")
    elif stats[1] > best_map_05:
        best_map_05 = stats[1]
        ct = 0
        torch.save({
                'epoch': 1,
                'model_state_dict': model.state_dict(),
                'data':  data_configs,
                'model_name': args['model'],
                }, f"{OUT_DIR}/best_model.pth")
    else: 
        ct += 1
        if ct > patience: 
            print('End: best val_map and best_map_05', best_map, best_map_05)
            break
            

        # Save loss plot for batch-wise list.
    save_loss_plot(OUT_DIR, train_loss_list)
    save_loss_plot( OUT_DIR,   train_loss_list_epoch,'epochs', 'train loss', save_name='train_loss_epoch')
    save_loss_plot(OUT_DIR, loss_cls_list, 'epochs',  'loss cls',save_name='train_loss_cls')
    save_loss_plot( OUT_DIR, loss_box_reg_list,  'epochs',  'loss bbox reg',save_name='train_loss_bbox_reg')
    save_loss_plot( OUT_DIR, loss_objectness_list, 'epochs', 'loss obj',save_name='train_loss_obj' )
    save_loss_plot( OUT_DIR,  loss_rpn_list, 'epochs','loss rpn bbox', save_name='train_loss_rpn_bbox')
    save_mAP(OUT_DIR, val_map_05, val_map)

    tensorboard_loss_log( 'Train loss',   np.array(train_loss_list_epoch),  writer,  epoch )
    tensorboard_map_log( name='mAP',  val_map_05=np.array(val_map_05), val_map=np.array(val_map), writer=writer,epoch=epoch)

    coco_log(OUT_DIR, stats)
    csv_log(OUT_DIR,  stats,  epoch, train_loss_list,loss_cls_list, loss_box_reg_list, loss_objectness_list, loss_rpn_list )


Epoch: [0]  [  0/180]  eta: 0:12:53  lr: 0.000007  loss: 1.8263 (1.8263)  loss_classifier: 0.7132 (0.7132)  loss_box_reg: 0.0478 (0.0478)  loss_objectness: 0.7206 (0.7206)  loss_rpn_box_reg: 0.3448 (0.3448)  time: 4.2966  data: 0.7697  max mem: 576
Epoch: [0]  [100/180]  eta: 0:00:19  lr: 0.001010  loss: 0.7125 (0.8876)  loss_classifier: 0.1385 (0.1734)  loss_box_reg: 0.1264 (0.1138)  loss_objectness: 0.2959 (0.4285)  loss_rpn_box_reg: 0.1346 (0.1720)  time: 0.2052  data: 0.0035  max mem: 576
Epoch: [0]  [179/180]  eta: 0:00:00  lr: 0.001000  loss: 0.7497 (0.8307)  loss_classifier: 0.1542 (0.1672)  loss_box_reg: 0.1456 (0.1299)  loss_objectness: 0.2668 (0.3726)  loss_rpn_box_reg: 0.1277 (0.1610)  time: 0.2035  data: 0.0034  max mem: 576
Epoch: [0] Total time: 0:00:40 (0.2276 s / it)


creating index...
index created!


Test:  [ 0/39]  eta: 0:00:25  model_time: 0.1937 (0.1937)  evaluator_time: 0.0399 (0.0399)  time: 0.6446  data: 0.3969  max mem: 576
Test:  [38/39]  eta: 0:00:00  model_time: 0.1226 (0.1249)  evaluator_time: 0.0499 (0.0613)  time: 0.1914  data: 0.0033  max mem: 576
Test: Total time: 0:00:08 (0.2056 s / it)


Averaged stats: model_time: 0.1226 (0.1249)  evaluator_time: 0.0499 (0.0613)
Accumulating evaluation results...
DONE (t=0.23s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.011
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.016
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.022
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.007
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.030
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.106
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.099
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium

Epoch: [1]  [  0/180]  eta: 0:01:52  lr: 0.001000  loss: 0.7648 (0.7648)  loss_classifier: 0.1812 (0.1812)  loss_box_reg: 0.1871 (0.1871)  loss_objectness: 0.2787 (0.2787)  loss_rpn_box_reg: 0.1180 (0.1180)  time: 0.6271  data: 0.3979  max mem: 576
Epoch: [1]  [100/180]  eta: 0:00:16  lr: 0.000999  loss: 0.7207 (0.7361)  loss_classifier: 0.1433 (0.1607)  loss_box_reg: 0.1466 (0.1655)  loss_objectness: 0.2603 (0.2690)  loss_rpn_box_reg: 0.1381 (0.1408)  time: 0.2071  data: 0.0036  max mem: 576
Epoch: [1]  [179/180]  eta: 0:00:00  lr: 0.000999  loss: 0.6452 (0.7225)  loss_classifier: 0.1494 (0.1593)  loss_box_reg: 0.1715 (0.1685)  loss_objectness: 0.2168 (0.2574)  loss_rpn_box_reg: 0.1102 (0.1372)  time: 0.2047  data: 0.0035  max mem: 576
Epoch: [1] Total time: 0:00:37 (0.2096 s / it)


...

Epoch: [53]  [  0/180]  eta: 0:02:15  lr: 0.000453  loss: 0.4977 (0.4977)  loss_classifier: 0.1068 (0.1068)  loss_box_reg: 0.1877 (0.1877)  loss_objectness: 0.1144 (0.1144)  loss_rpn_box_reg: 0.0888 (0.0888)  time: 0.7518  data: 0.5111  max mem: 576
Epoch: [53]  [100/180]  eta: 0:00:18  lr: 0.000444  loss: 0.5001 (0.4904)  loss_classifier: 0.1129 (0.1115)  loss_box_reg: 0.1885 (0.1866)  loss_objectness: 0.1086 (0.1108)  loss_rpn_box_reg: 0.0717 (0.0816)  time: 0.2207  data: 0.0038  max mem: 576
Epoch: [53]  [179/180]  eta: 0:00:00  lr: 0.000437  loss: 0.5205 (0.4978)  loss_classifier: 0.1209 (0.1123)  loss_box_reg: 0.1860 (0.1895)  loss_objectness: 0.1157 (0.1119)  loss_rpn_box_reg: 0.0896 (0.0841)  time: 0.2192  data: 0.0036  max mem: 576
Epoch: [53] Total time: 0:00:40 (0.2264 s / it)


creating index...
index created!


Test:  [ 0/39]  eta: 0:00:40  model_time: 0.1278 (0.1278)  evaluator_time: 0.0131 (0.0131)  time: 1.0287  data: 0.8721  max mem: 576
Test:  [38/39]  eta: 0:00:00  model_time: 0.1352 (0.1464)  evaluator_time: 0.0244 (0.0344)  time: 0.1912  data: 0.0032  max mem: 576
Test: Total time: 0:00:08 (0.2126 s / it)


Averaged stats: model_time: 0.1352 (0.1464)  evaluator_time: 0.0244 (0.0344)
Accumulating evaluation results...
DONE (t=0.10s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.122
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.364
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.052
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.107
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.174
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.212
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.067
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.219
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.317
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.285
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium

Epoch: [54]  [  0/180]  eta: 0:02:16  lr: 0.000437  loss: 0.7019 (0.7019)  loss_classifier: 0.1537 (0.1537)  loss_box_reg: 0.2491 (0.2491)  loss_objectness: 0.1611 (0.1611)  loss_rpn_box_reg: 0.1380 (0.1380)  time: 0.7575  data: 0.5114  max mem: 576


In [10]:
!python eval.py --data data_configs/land4sensor.yaml --weights outputs/training/land4sensor_training/best_model.pth --imgsz 224 --device cuda:0  --model fasterrcnn_swin

building models from outputs/training/land4sensor_training/best_model.pth
current model setting: base_from_mae
Checking Labels and images...
100%|█████████████████████████████████████| 309/309 [00:00<00:00, 254075.66it/s]
creating index...
index created!
Averaged stats: model_time: 0.1159 (0.1405)  evaluator_time: 0.0249 (0.0356)
Accumulating evaluation results...
DONE (t=0.10s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.127
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.364
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.052
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.112
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.175
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.180
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.064
 Average Recall     (AR) @[ IoU=0.50:0.9

In [10]:
!python inference.py --input /hdd/yuchen/pipeline/training_pipeline/data_land4sensor/valid --data data_configs/land4sensor.yaml --weights outputs/training/land4sensor_training/best_model.pth --imgsz 224 --device cuda:0 --threshold 0.8 --model fasterrcnn_swin --save_img True 

Building from model name arguments...
current model setting: base_from_mae
Test instances: 309
Image 1 done...
--------------------------------------------------
Image 2 done...
--------------------------------------------------
Image 3 done...
--------------------------------------------------
Image 4 done...
--------------------------------------------------
Image 5 done...
--------------------------------------------------
Image 6 done...
--------------------------------------------------
Image 7 done...
--------------------------------------------------
Image 8 done...
--------------------------------------------------
Image 9 done...
--------------------------------------------------
Image 10 done...
--------------------------------------------------
Image 11 done...
--------------------------------------------------
Image 12 done...
--------------------------------------------------
Image 13 done...
--------------------------------------------------
Image 14 done...
-------------